In [1]:
from modules.setup import *
from modules.utils import *
from modules.MetaNorm3D import *
from modules.MetaAlign3D import *
from modules.MetaImpute3D import *
from modules.MetaInterp3D import *
from modules.MetaAtlas3D import *
from modules.visualize import *
from modules.evaluate import *

In [2]:
group = 'wt' # 'wt','wt2','gaa','5x'
omic = 'lipid' # 'lipid', 'sm'

In [3]:
# input data path
data_file_path = f'./data/{group}/3d_brain_{group}_pbp_{omic}_preprocessed.csv'
shell_file_path = f'./data/{group}/{group}_shell.csv' # shell is the off-tissue matrix, optional

# Load data

In [5]:
df = pd.read_csv(data_file_path,index_col=0,usecols=range(31)) # only load first 30 columns for demonstration

In [ ]:
shell = pd.read_csv(shell_file_path,index_col=0)
df = remove_shell(df,shell) # remove shell from tissue, optional step

In [ ]:
df.head()

In [9]:
df, deleted_compound = delete_low_prevalence_compound(df, 0.1, first_feature='LPA.18.1.') # LPA.18.1 is the first compound

In [ ]:
deleted_compound

# Preview

## check orientation

In [ ]:
preview_matrix = create_compound_matrix(df, compound='LPA.18.1.',reverse=True)
display_montage(preview_matrix,grayscale=False,cmap=new_cmap1)

In [12]:
# flip in the up/down direction
df = flip_axis(df,flipud=True)

In [ ]:
preview_matrix = create_compound_matrix(df, compound='LPA.18.1.',reverse=True)
display_montage(preview_matrix,grayscale=False,cmap=new_cmap1)

## remove inferior tissue

In [14]:
# delete inferior sections
inferior_tissues_id = [62,67,116]

In [15]:
df = delete_inferior_tissues(df,inferior_tissues_id,first_feature='LPA.18.1.',reverse=True)

In [ ]:
preview_matrix = create_compound_matrix(df, compound='LPA.18.1.',reverse=True)
display_montage(preview_matrix,grayscale=False,cmap=new_cmap1)

# MetaNorm3D

In [17]:
# normalization
meta_norm = MetaNorm3D(df, first_feature='LPA.18.1.')

In [18]:
# totalsum normalization
data = meta_norm.totalsum_norm()

In [ ]:
preview_matrix = create_compound_matrix(data, compound='LPA.18.1.',reverse=True)
display_montage(preview_matrix,grayscale=False,cmap=new_cmap1)

In [20]:
# section normalization
data = meta_norm.section_norm()

In [ ]:
preview_matrix = create_compound_matrix(data, compound='LPA.18.1.',reverse=True)
display_montage(preview_matrix,grayscale=False,cmap=new_cmap1)

# MetaAlign3D

In [22]:
compound = 'LPA.18.1.'

In [ ]:
meta_align = MetaAlign3D(group,data,compound,first_feature='LPA.18.1.', reverse=True)

## Manual slices fitting

In [ ]:
# manual slices fitting and create compound matrix
compound_matrix = meta_align.create_compound_matrix()

In [ ]:
display_montage(compound_matrix,grayscale=True)

In [ ]:
display_animation(compound_matrix,grayscale=False,cmap=new_cmap1)

## Sequential Alignment

In [ ]:
aligned_matrix = meta_align.seq_align()

In [ ]:
display_animation(aligned_matrix,grayscale=False,cmap=new_cmap1)

# MetaImpute3D

In [29]:
meta_impute = MetaImpute3D(aligned_matrix,radius=1)

In [30]:
imputed_matrix = meta_impute.seq_impute()

In [ ]:
display_montage(imputed_matrix,grayscale=False,cmap=new_cmap1)

In [ ]:
display_animation(imputed_matrix,grayscale=False,cmap=new_cmap1)

# MetaInterp3D

In [33]:
# add 2 more slices 
meta_interp = MetaInterp3D(imputed_matrix,2)

In [34]:
interpolated_matrix = meta_interp.interp()

In [ ]:
display_interp_montage(interpolated_matrix,new_cmap1,new_cmap2,insert=2)

# MetaAtlas3D

In [36]:
meta_atlas = MetaAtlas3D(interpolated_matrix,resolution=50,thickness=30,gap=0,insert=2)
nii_img = meta_atlas.create_nii()

In [38]:
nii_path = f'./output/{group}/{omic}/3D'
if not os.path.exists(nii_path):
    os.makedirs(nii_path)
nib.save(nii_img, f'{nii_path}/{compound}.nii.gz')